In [1]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier,DecisionTreeRegressor,export_graphviz
import graphviz
import dtreeviz
import pandas as pd
import numpy as np
import pickle
import os
import json
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor,RandomForestClassifier
from sklearn.metrics import accuracy_score,confusion_matrix
from sklearn import tree
from pypinyin import pinyin, lazy_pinyin, Style
from autogluon.tabular import TabularDataset, TabularPredictor

In [53]:
# autogluon保存路径
model_path = r"F:\cache_data\zone_ana\sb\modle\autogluon_type_best"
# 检查路径是否存在，否则便创建
if not os.path.exists(model_path):
    os.makedirs(model_path)
else:
    print("文件夹已存在")

In [3]:
dataset = pd.read_csv(r"F:\cache_data\zone_ana\sb\train_data\soil_type_point.csv")

In [ ]:
# # 筛选满足条件的记录
# pr = dataset[(dataset['MZMC'] == "河流冲积物") & (dataset['NEW_TZ'] == "中层砂质灰潮土")]

# # 如果筛选出的记录少于 5 个，我们将修改所有记录
# num_to_change = min(5, len(pr))

# if num_to_change > 0:
#     # 随机选择 num_to_change 个索引
#     random_indices = np.random.choice(pr.index, num_to_change, replace=False)
#     # 修改选中记录的 NEW_TZ 值
#     dataset.loc[random_indices, 'NEW_TZ'] = "厚层砂质灰潮土"

#     print(f"已随机修改 {num_to_change} 条记录的 NEW_TZ 值为 '厚层砂质灰潮土'")
# else:
#     print("没有找到满足条件的记录")

# # 验证修改结果
# modified_records = dataset[(dataset['MZMC'] == "河流冲积物") & (dataset['NEW_TZ'] == "厚层砂质灰潮土")]
# print(f"修改后，符合新条件的记录数：{len(modified_records)}")

In [4]:
dataset["TZ_label"] = dataset.NEW_TZ.astype("category").cat.codes

In [5]:
dataset['DLMCN'] = dataset['DLMCN'].astype('category')
dataset['MZMC'] = dataset['MZMC'].astype('category')
dataset['TZ_label'] = dataset['TZ_label'].astype('category')

In [6]:
result = dataset.groupby('TZ_label', observed=True)["NEW_TZ"].apply(lambda x: list(x.unique())).to_dict()
for one_type in result:
    result[one_type] = result[one_type][0]
print(result)

{0: '中层壤质中性紫色土', 1: '中层壤质黄色石灰土', 2: '中层泥质黄壤', 3: '中层灰泥质黄壤', 4: '中层砂泥质黄壤', 5: '中层砂质灰潮土', 6: '中层硅质黄壤', 7: '中层红泥质黄壤', 8: '中层黏质黄色石灰土', 9: '厚层壤质中性紫色土', 10: '厚层壤质黄色石灰土', 11: '厚层泥质黄壤', 12: '厚层灰泥质黄壤', 13: '厚层砂泥质黄壤', 14: '厚层砂质灰潮土', 15: '厚层硅质黄壤', 16: '厚层红泥质黄壤', 17: '厚层黏质黄色石灰土', 18: '浅石灰泥田', 19: '渗石灰泥田', 20: '渗紫泥田', 21: '漂石灰泥田', 22: '潮泥田', 23: '潮泥砂田', 24: '石灰泥田', 25: '紫泥田', 26: '腐中层壤质中性紫色土', 27: '腐中层壤质酸性紫色土', 28: '腐中层壤质黄色石灰土', 29: '腐中层泥质黄壤', 30: '腐中层泥质黄棕壤', 31: '腐中层灰泥质黄壤', 32: '腐中层砂泥质黄壤', 33: '腐中层硅质黄壤', 34: '腐中层硅质黄棕壤', 35: '腐中层红泥质黄壤', 36: '腐中层黏质中性紫色土', 37: '腐中层黏质黄色石灰土', 38: '腐厚层壤质中性紫色土', 39: '腐厚层壤质黄色石灰土', 40: '腐厚层泥质黄壤', 41: '腐厚层泥质黄棕壤', 42: '腐厚层灰泥质黄壤', 43: '腐厚层砂泥质黄壤', 44: '腐厚层硅质黄壤', 45: '腐厚层红泥质黄壤', 46: '腐厚层黏质黄色石灰土', 47: '腐薄层壤质中性紫色土', 48: '腐薄层壤质酸性紫色土', 49: '腐薄层壤质黄色石灰土', 50: '腐薄层泥质黄壤', 51: '腐薄层泥质黄棕壤', 52: '腐薄层灰泥质石灰性粗骨土', 53: '腐薄层灰泥质黄壤', 54: '腐薄层砂泥质黄壤', 55: '腐薄层硅质酸性粗骨土', 56: '腐薄层硅质黄壤', 57: '腐薄层硅质黄棕壤', 58: '腐薄层红泥质黄壤', 59: '腐薄层黏质黄色石灰土', 60: '薄层壤质中性紫色土', 61: '薄层壤质酸性紫色土', 62: '薄层壤质黄色石灰土', 63: '薄层泥质酸性粗骨土'

In [7]:
# 保存映射关系
with open(r'D:\worker_code\Terrain_Test\data\soil_dict_20241029_sb.json', 'w') as f:
    json.dump(result, f)

In [8]:
# 查看地类类别
tdlylx_df = pd.DataFrame(dataset['TDLYLX'].value_counts())
tdlylx_df

,count
TDLYLX,
自然土,5610
非自然土,1508
水稻土,1138


In [9]:
# 查看地类类别
dl_df = pd.DataFrame(dataset['DLMCN'].value_counts())
dl_df

,count
DLMCN,
乔木林地,4333
旱地,1280
灌木林地,1223
水田,1138
天然牧草地,105
果园,82
其他林地,46
其他草地,30
其他园地,10


In [10]:
# 查看母质类别
mz_df = pd.DataFrame(dataset['MZMC'].value_counts())
mz_df

,count
MZMC,
白云岩,4340
灰岩,851
砂岩,786
砂页岩,649
第四纪粘土,605
泥(页)岩,363
紫色砂页岩,258
河流冲积物,186
板岩,108


### 砂岩、变余砂岩土种

In [11]:
def delete_row(df, mzmc, new_tz):
    query_result = df[(df['MZMC'] == f'{mzmc}') & (df['NEW_TZ'] == f'{new_tz}')]
    df = df.drop(query_result.index)
    print(query_result.shape[0])
    return df


In [12]:
# 筛选砂岩类水稻土数据
sy_sdt_data = dataset[(dataset['TDLYLX']=='水稻土')
                      & ((dataset['MZMC'] == '砂岩')|(dataset['MZMC'] == '变余砂岩'))]
pd.unique(sy_sdt_data['NEW_TZ']),pd.unique(sy_sdt_data['NEW_TS'])

(array(['黄渗白粉泥田', '黄白粉泥田', '黄浅白粉泥田'], dtype=object),
 array(['渗白粉泥田', '白粉泥田', '浅白粉泥田'], dtype=object))

In [13]:
# 筛砂岩选非自然土数据
sy_fzrt_data = dataset[(dataset['TDLYLX']=='非自然土') 
                       & ((dataset['MZMC'] == '砂岩')|(dataset['MZMC'] == '变余砂岩'))]
pd.unique(sy_fzrt_data['NEW_TZ']),pd.unique(sy_fzrt_data['NEW_TS'])

(array(['薄层硅质黄壤', '厚层硅质黄壤', '中层硅质黄壤'], dtype=object),
 array(['硅质黄壤'], dtype=object))

In [14]:
# 筛选自然土数据
sy_zrt_data = dataset[(dataset['TDLYLX']=='自然土')
                   & ((dataset['MZMC'] == '砂岩')|(dataset['MZMC'] == '变余砂岩'))]
pd.unique(sy_zrt_data['NEW_TZ']),pd.unique(sy_zrt_data['NEW_TS'])

(array(['腐薄层硅质黄壤', '腐中层硅质黄壤', '腐厚层硅质黄壤', '腐薄层硅质黄棕壤', '腐中层硅质黄棕壤',
        '腐薄层硅质酸性粗骨土'], dtype=object),
 array(['硅质黄壤', '硅质暗黄棕壤', '硅质酸性粗骨土'], dtype=object))

### 碳酸岩土种

In [15]:
# 筛选碳酸岩水稻土数据
tsy_sdt_data = dataset[(dataset['TDLYLX']=='水稻土')
                      & ((dataset['MZMC'] == '白云岩')|(dataset['MZMC'] == '灰岩'))]
pd.unique(tsy_sdt_data['NEW_TZ']),pd.unique(tsy_sdt_data['NEW_TS']),

(array(['石灰泥田', '浅石灰泥田', '渗石灰泥田', '黄灰泥田', '黄渗灰泥田', '黄石灰泥田', '黄浅灰泥田',
        '漂石灰泥田'], dtype=object),
 array(['灰泥田', '浅灰泥田', '渗灰泥田', '黄灰泥田', '漂灰泥田'], dtype=object))

In [16]:
# 筛选碳酸岩选非自然土数据
tsy_fzrt_data = dataset[(dataset['TDLYLX']=='非自然土')
                       & ((dataset['MZMC'] == '白云岩')|(dataset['MZMC'] == '灰岩'))]

pd.unique(tsy_fzrt_data['NEW_TZ']),pd.unique(tsy_fzrt_data['NEW_TS']),

(array(['中层壤质黄色石灰土', '薄层壤质黄色石灰土', '厚层壤质黄色石灰土', '薄层灰泥质黄壤', '中层灰泥质黄壤',
        '中层黏质黄色石灰土', '厚层灰泥质黄壤', '厚层黏质黄色石灰土', '薄层黏质黄色石灰土', '薄层灰泥质石灰性粗骨土'],
       dtype=object),
 array(['壤质黄色石灰土', '灰泥质黄壤', '黏质黄色石灰土', '灰泥质石灰性粗骨土'], dtype=object))

In [17]:
# 筛选碳酸岩自然土数据

tsy_zrt_data = dataset[(dataset['TDLYLX']=='自然土')
                       & ((dataset['MZMC'] == '白云岩')|(dataset['MZMC'] == '灰岩'))]
pd.unique(tsy_zrt_data['NEW_TZ']),pd.unique(tsy_zrt_data['NEW_TS']),

(array(['腐中层壤质黄色石灰土', '腐薄层壤质黄色石灰土', '腐厚层壤质黄色石灰土', '腐中层灰泥质黄壤', '腐薄层灰泥质黄壤',
        '腐厚层灰泥质黄壤', '腐中层黏质黄色石灰土', '腐薄层黏质黄色石灰土', '腐厚层黏质黄色石灰土',
        '腐薄层灰泥质石灰性粗骨土'], dtype=object),
 array(['壤质黄色石灰土', '灰泥质黄壤', '黏质黄色石灰土', '灰泥质石灰性粗骨土'], dtype=object))

### 第四系红粘土

In [18]:
# 筛选第四系红粘土水稻土数据
hnt_sdt_data = dataset[(dataset['TDLYLX']=='水稻土')
                       & ((dataset['MZMC'] == '第四纪红黏土')|(dataset['MZMC'] == '第四纪粘土'))]
pd.unique(hnt_sdt_data['NEW_TZ']),pd.unique(hnt_sdt_data['NEW_TS']),

(array(['黄红泥田', '黄渗红泥田'], dtype=object), array(['红泥田', '渗红泥田'], dtype=object))

In [19]:
# 筛选第四系红粘土非自然土数据
hnt_fzrt_data = dataset[(dataset['TDLYLX']=='非自然土')
                       & ((dataset['MZMC'] == '第四纪红黏土')|(dataset['MZMC'] == '第四纪粘土'))]
pd.unique(hnt_fzrt_data['NEW_TZ']),pd.unique(hnt_fzrt_data['NEW_TS']),

(array(['中层红泥质黄壤', '薄层红泥质黄壤', '厚层红泥质黄壤'], dtype=object),
 array(['红泥质黄壤'], dtype=object))

In [20]:
# 筛选第四系红粘土自然土数据
hnt_zrt_data = dataset[(dataset['TDLYLX']=='自然土')
                       & ((dataset['MZMC'] == '第四纪红黏土')|(dataset['MZMC'] == '第四纪粘土'))]
pd.unique(hnt_zrt_data['NEW_TZ']),pd.unique(hnt_zrt_data['NEW_TS']),

(array(['腐中层红泥质黄壤', '腐薄层红泥质黄壤', '腐厚层红泥质黄壤'], dtype=object),
 array(['红泥质黄壤'], dtype=object))

### 泥(页)岩

In [21]:
# 筛选泥(页)岩水稻土数据
nyy_sdt_data = dataset[(dataset['TDLYLX']=='水稻土')
                       & ((dataset['MZMC'] == '泥(页)岩')|(dataset['MZMC'] == '板岩'))]
pd.unique(nyy_sdt_data['NEW_TZ']),pd.unique(nyy_sdt_data['NEW_TS']),

(array(['黄鳝泥田', '黄渗鳝泥田', '黄浅鳝泥田'], dtype=object),
 array(['鳝泥田', '渗鳝泥田', '浅鳝泥田'], dtype=object))

In [22]:
# 筛选泥(页)岩非自然土数据
nyy_fzrt_data = dataset[(dataset['TDLYLX']=='非自然土')
                       & ((dataset['MZMC'] == '泥(页)岩')|(dataset['MZMC'] == '板岩'))]
pd.unique(nyy_fzrt_data['NEW_TZ']),pd.unique(nyy_fzrt_data['NEW_TS']),

(array(['中层泥质黄壤', '薄层泥质黄壤', '厚层泥质黄壤', '薄层泥质黄棕壤', '薄层泥质酸性粗骨土'], dtype=object),
 array(['泥质黄壤', '泥质暗黄棕壤', '泥质酸性粗骨土'], dtype=object))

In [23]:
# 筛选泥(页)岩自然土数据
nyy_zrt_data = dataset[(dataset['TDLYLX']=='自然土')
                       & ((dataset['MZMC'] == '泥(页)岩')|(dataset['MZMC'] == '板岩'))]
pd.unique(nyy_zrt_data['NEW_TZ']),pd.unique(nyy_zrt_data['NEW_TS']),

(array(['腐中层泥质黄壤', '腐薄层泥质黄壤', '腐厚层泥质黄壤', '腐厚层泥质黄棕壤', '腐薄层泥质黄棕壤',
        '腐中层泥质黄棕壤'], dtype=object),
 array(['泥质黄壤', '泥质暗黄棕壤'], dtype=object))

### 紫红色砂页岩	

In [24]:
# 筛选紫红色砂页岩水稻土数据
zhsyy_sdt_data = dataset[(dataset['TDLYLX']=='水稻土')
                       & ((dataset['MZMC'] == '酸性紫色砂页岩')|(dataset['MZMC'] == '紫色砂页岩'))]
pd.unique(zhsyy_sdt_data['NEW_TZ']),pd.unique(zhsyy_sdt_data['NEW_TS']),

(array(['紫泥田', '渗紫泥田', '酸紫泥田', '酸渗紫泥田'], dtype=object),
 array(['紫泥田', '渗紫泥田'], dtype=object))

In [25]:
# 筛选紫红色砂页岩非自然土数据
zhsyy_fzrt_data = dataset[(dataset['TDLYLX']=='非自然土')
                       & ((dataset['MZMC'] == '酸性紫色砂页岩')|(dataset['MZMC'] == '紫色砂页岩'))]
pd.unique(zhsyy_fzrt_data['NEW_TZ']),pd.unique(zhsyy_fzrt_data['NEW_TS']),

(array(['中层壤质中性紫色土', '厚层壤质中性紫色土', '薄层壤质中性紫色土', '薄层壤质酸性紫色土'], dtype=object),
 array(['壤质中性紫色土', '壤质酸性紫色土', '黏质中性紫色土'], dtype=object))

In [26]:
# 筛选紫红色砂页岩自然土数据
zhsyy_zrt_data = dataset[(dataset['TDLYLX']=='自然土')
                       & ((dataset['MZMC'] == '酸性紫色砂页岩')|(dataset['MZMC'] == '紫色砂页岩'))]
pd.unique(zhsyy_zrt_data['NEW_TZ']),pd.unique(zhsyy_zrt_data['NEW_TS']),

(array(['腐中层壤质中性紫色土', '腐薄层壤质中性紫色土', '腐薄层壤质酸性紫色土', '腐厚层壤质中性紫色土',
        '腐中层壤质酸性紫色土', '腐中层黏质中性紫色土'], dtype=object),
 array(['壤质中性紫色土', '壤质酸性紫色土', '黏质中性紫色土'], dtype=object))

### 河流冲积物

In [27]:
# 筛选河流冲积物水稻土数据
hlcjw_sdt_data = dataset[(dataset['TDLYLX']=='水稻土')
                       & (dataset['MZMC'] == '河流冲积物')]
pd.unique(hlcjw_sdt_data['NEW_TZ']),pd.unique(hlcjw_sdt_data['NEW_TS']),

(array(['潮泥砂田', '潮泥田'], dtype=object), array(['潮泥砂田', '潮泥田'], dtype=object))

In [28]:
# 筛选河流冲积物非自然土数据
hlcjw_fzrt_data = dataset[(dataset['TDLYLX']=='非自然土')
                       & (dataset['MZMC'] == '河流冲积物')]

pd.unique(hlcjw_fzrt_data['NEW_TZ']),pd.unique(hlcjw_fzrt_data['NEW_TS']),

(array(['中层砂质灰潮土', '厚层砂质灰潮土'], dtype=object), array(['砂质灰潮土'], dtype=object))

### 砂页岩

In [29]:
# 筛选砂页岩水稻土数据
syy_sdt_data = dataset[(dataset['TDLYLX']=='水稻土')
                       & ((dataset['MZMC'] == '砂页岩')|(dataset['MZMC'] == '砂砾岩'))]
pd.unique(syy_sdt_data['NEW_TZ']),pd.unique(syy_sdt_data['NEW_TS']),

(array(['黄渗砂泥田', '黄砂泥田', '黄浅砂泥田'], dtype=object),
 array(['渗砂泥田', '砂泥田', '浅砂泥田'], dtype=object))

In [30]:
# 筛选砂页岩非自然土数据
syy_fzrt_data = dataset[(dataset['TDLYLX']=='非自然土')
                       & ((dataset['MZMC'] == '砂页岩')|(dataset['MZMC'] == '砂砾岩'))]
pd.unique(syy_fzrt_data['NEW_TZ']),pd.unique(syy_fzrt_data['NEW_TS']),

(array(['中层砂泥质黄壤', '厚层砂泥质黄壤', '薄层砂泥质黄壤'], dtype=object),
 array(['砂泥质黄壤'], dtype=object))

In [31]:
# 筛选砂页岩物自然土数据
syy_zrt_data = dataset[(dataset['TDLYLX']=='自然土')
                       & ((dataset['MZMC'] == '砂页岩')|(dataset['MZMC'] == '砂砾岩'))]
pd.unique(syy_zrt_data['NEW_TZ']),pd.unique(syy_zrt_data['NEW_TS']),

(array(['腐中层砂泥质黄壤', '腐薄层砂泥质黄壤', '腐厚层砂泥质黄壤'], dtype=object),
 array(['砂泥质黄壤'], dtype=object))

In [32]:
columns_List = [_ for _ in dataset.columns]

In [33]:
columns_List

['point_id',
 'longitude',
 'latitude',
 'TL',
 'YL',
 'TS',
 'TZ',
 'label',
 'LON',
 'LAT',
 'MZMC',
 'DLDMN',
 'DLMCN',
 'NEW_TL',
 'NEW_YL',
 'NEW_TS',
 'NEW_TZ',
 'TDLYLX',
 'analytical_hillshading',
 'aspect',
 'channel_network_base_level',
 'channel_network_distance',
 'closed_depressions',
 'convergence_index',
 'dem',
 'dl',
 'dz',
 'etp2022',
 'etp2022_1',
 'etp2022_10',
 'etp2022_11',
 'etp2022_12',
 'etp2022_2',
 'etp2022_3',
 'etp2022_4',
 'etp2022_5',
 'etp2022_6',
 'etp2022_7',
 'etp2022_8',
 'etp2022_9',
 'etp2022_mean',
 'evi',
 'lat',
 'lon',
 'lswi',
 'ls_factor',
 'mndwi',
 'mrrtf',
 'mrvbf',
 'ndmi',
 'ndvi',
 'ndwi',
 'night2022',
 'pca_1',
 'pca_2',
 'plan_curvature',
 'pre2022',
 'pre2022_1',
 'pre2022_10',
 'pre2022_11',
 'pre2022_12',
 'pre2022_2',
 'pre2022_3',
 'pre2022_4',
 'pre2022_5',
 'pre2022_6',
 'pre2022_7',
 'pre2022_8',
 'pre2022_9',
 'pre2022_mean',
 'profile_curvature',
 'relative_slope_position',
 'savi',
 'slope',
 'slope_postion_101',
 'slope_p

In [37]:
features_use = ['analytical_hillshading','aspect','channel_network_base_level','channel_network_distance',
 'convergence_index','dem','dz','dl','etp2022_1','etp2022_10','etp2022_11','etp2022_12','etp2022_2','etp2022_3',
 'etp2022_4','etp2022_5','etp2022_6','etp2022_7','etp2022_8','etp2022_9','etp2022_mean','evi','lat','lon','lswi',
 'ls_factor','mndwi','mrrtf','mrvbf','ndmi','ndvi','ndwi','night2022','pca_1','pca_2','plan_curvature',
 'pre2022_1','pre2022_10','pre2022_11','pre2022_12','pre2022_2','pre2022_3','pre2022_4','pre2022_5','pre2022_6',
 'pre2022_7','pre2022_8','pre2022_9','pre2022_mean','profile_curvature','relative_slope_position','savi','slope',
 'slope_postion_101_smooth','tmp2022_1','tmp2022_10','tmp2022_11','tmp2022_12',
 'tmp2022_2','tmp2022_3','tmp2022_4','tmp2022_5','tmp2022_6','tmp2022_7','tmp2022_8','tmp2022_9','tmp2022_mean',
 'topographic_wetness_index','total_catchment_area','valley_depth','vari']

In [38]:

sdt_features = features_use
fzrt_features = features_use
zrt_features = features_use
target = "TZ_label"

### 砂岩数据集

In [39]:
# 获取sdt数据集
sy_sdt_data = sy_sdt_data[sdt_features+[target]]
# 获取非自然土数据集
sy_fzrt_data = sy_fzrt_data[fzrt_features+[target]]
# 获取自然土数据集
sy_zrt_data = sy_zrt_data[zrt_features+[target]]

In [ ]:
sy_sdt_data.columns,len(sy_sdt_data.columns)

### 碳酸岩数据集

In [41]:
# 获取sdt数据集
tsy_sdt_data = tsy_sdt_data[sdt_features+[target]]
# 获取非自然土数据集
tsy_fzrt_data = tsy_fzrt_data[fzrt_features+[target]]
# 获取自然土数据集
tsy_zrt_data = tsy_zrt_data[zrt_features+[target]]

### 第四系红粘土数据集

In [42]:
# 获取sdt数据集
hnt_sdt_data = hnt_sdt_data[sdt_features+[target]]
# 获取非自然土数据集
hnt_fzrt_data = hnt_fzrt_data[fzrt_features+[target]]
# 获取自然土数据集
hnt_zrt_data = hnt_zrt_data[zrt_features+[target]]

### 泥(页)岩数据集

In [43]:
# 获取sdt数据集
nyy_sdt_data = nyy_sdt_data[sdt_features+[target]]
# 获取非自然土数据集
nyy_fzrt_data = nyy_fzrt_data[fzrt_features+[target]]
# 获取自然土数据集
nyy_zrt_data = nyy_zrt_data[zrt_features+[target]]

### 紫红色砂页岩数据集

In [44]:
# 获取sdt数据集
zhsyy_sdt_data = zhsyy_sdt_data[sdt_features+[target]]
# 获取非自然土数据集
zhsyy_fzrt_data = zhsyy_fzrt_data[fzrt_features+[target]]
# 获取自然土数据集
zhsyy_zrt_data = zhsyy_zrt_data[zrt_features+[target]]

### 河流冲积物数据集

In [45]:
# 获取sdt数据集
hlcjw_sdt_data = hlcjw_sdt_data[sdt_features+[target]]
# 获取非自然土
hlcjw_fzrt_data = hlcjw_fzrt_data[sdt_features+[target]]
# 获取非自然土

### 砂页岩数据集

In [46]:
# 获取sdt数据集
syy_sdt_data = syy_sdt_data[sdt_features+[target]]
# 获取非自然土数据集
syy_fzrt_data = syy_fzrt_data[fzrt_features+[target]]
# 获取自然土数据集
syy_zrt_data = syy_zrt_data[zrt_features+[target]]

In [49]:
# 查看各个数据集的大小
print('砂岩',sy_sdt_data.shape,sy_fzrt_data.shape,sy_zrt_data.shape)
print('碳酸岩',tsy_sdt_data.shape,tsy_fzrt_data.shape,tsy_zrt_data.shape)
print('第四系红粘土',hnt_sdt_data.shape,hnt_fzrt_data.shape,hnt_zrt_data.shape)
print('泥页岩',nyy_sdt_data.shape,nyy_fzrt_data.shape,nyy_zrt_data.shape)
print('紫红色砂页岩',zhsyy_sdt_data.shape,zhsyy_fzrt_data.shape,zhsyy_zrt_data.shape)
print('河流冲积物',hlcjw_sdt_data.shape,hlcjw_fzrt_data.shape)
print('砂页岩',syy_sdt_data.shape,syy_fzrt_data.shape,syy_zrt_data.shape)

# 计算总数
total_data = pd.concat([sy_sdt_data,sy_fzrt_data,sy_zrt_data,tsy_sdt_data,tsy_fzrt_data,tsy_zrt_data,hnt_sdt_data,hnt_fzrt_data,hnt_zrt_data,nyy_sdt_data,nyy_fzrt_data,nyy_zrt_data,zhsyy_sdt_data,zhsyy_fzrt_data,zhsyy_zrt_data,hlcjw_sdt_data,hlcjw_fzrt_data,syy_sdt_data,syy_fzrt_data,syy_zrt_data])
total_data.shape,dataset.shape[0]


砂岩 (120, 72) (75, 72) (619, 72)
碳酸岩 (372, 72) (833, 72) (3986, 72)
第四系红粘土 (137, 72) (336, 72) (147, 72)
泥页岩 (95, 72) (85, 72) (291, 72)
紫红色砂页岩 (86, 72) (98, 72) (133, 72)
河流冲积物 (166, 72) (20, 72)
砂页岩 (162, 72) (61, 72) (434, 72)


((8256, 72), 8256)

In [50]:
# 确定标签
label = "TZ_label"
eval_metric = 'f1_weighted'
problem_type = 'multiclass'
# # 10折交叉验证
# cv_num = 10
# 初始化模型和超参数
hyperparameters={
	'NN_TORCH': {},
	'FASTAI': {},
	'RF': [{'criterion': 'gini', 'ag_args': {'name_suffix': 'Gini', 'problem_types': ['binary', 'multiclass']}}, {'criterion': 'entropy', 'ag_args': {'name_suffix': 'Entr', 'problem_types': ['binary', 'multiclass']}}, {'criterion': 'squared_error', 'ag_args': {'name_suffix': 'MSE', 'problem_types': ['regression', 'quantile']}}],
	'XT': [{'criterion': 'gini', 'ag_args': {'name_suffix': 'Gini', 'problem_types': ['binary', 'multiclass']}}, {'criterion': 'entropy', 'ag_args': {'name_suffix': 'Entr', 'problem_types': ['binary', 'multiclass']}}, {'criterion': 'squared_error', 'ag_args': {'name_suffix': 'MSE', 'problem_types': ['regression', 'quantile']}}]
}


### 模型训练

In [71]:
def train_model(train_data,label,eval_metric,hyperparameters,problem_type,model_master,model_branch,model_root):
    # 训练model
    temp_master_path =os.path.join(model_root,model_master)
    temp_branch_path = os.path.join(temp_master_path,model_branch)
    # 检查路径是否存在，否则便创建
    if not os.path.exists(temp_branch_path):
        os.makedirs(temp_branch_path)
    else:
        print("文件夹已存在")
    # 执行训练
    train_predictor = TabularPredictor(label=label,path=temp_branch_path,eval_metric=eval_metric,problem_type=problem_type).fit(train_data,hyperparameters=hyperparameters)
    train_predictor.fit_summary()
    return train_predictor.model_best


### 模型训练

In [72]:
# 砂岩
sy_sdt_predictor = train_model(sy_sdt_data,label=label,eval_metric=eval_metric,hyperparameters=hyperparameters,problem_type=problem_type,model_master='sy',model_branch='sdt',model_root=model_path) 
sy_fzrt_predictor = train_model(sy_fzrt_data,label=label,eval_metric=eval_metric,hyperparameters=hyperparameters,problem_type=problem_type,model_master='sy',model_branch='fzrt',model_root=model_path) 
sy_zrt_predictor = train_model(sy_zrt_data,label=label,eval_metric=eval_metric,hyperparameters=hyperparameters,problem_type=problem_type,model_master='sy',model_branch='zrt',model_root=model_path) 
# 碳酸岩
tsy_sdt_predictor = train_model(tsy_sdt_data,label=label,eval_metric=eval_metric,hyperparameters=hyperparameters,problem_type=problem_type,model_master='tsy',model_branch='sdt',model_root=model_path) 
tsy_fzrt_predictor = train_model(tsy_fzrt_data,label=label,eval_metric=eval_metric,hyperparameters=hyperparameters,problem_type=problem_type,model_master='tsy',model_branch='fzrt',model_root=model_path) 
tsy_zrt_predictor = train_model(tsy_zrt_data,label=label,eval_metric=eval_metric,hyperparameters=hyperparameters,problem_type=problem_type,model_master='tsy',model_branch='zrt',model_root=model_path) 
# 第四系红粘土
hnt_sdt_predictor = train_model(hnt_sdt_data,label=label,eval_metric=eval_metric,hyperparameters=hyperparameters,problem_type=problem_type,model_master='hnt',model_branch='sdt',model_root=model_path) 
hnt_fzrt_predictor = train_model(hnt_fzrt_data,label=label,eval_metric=eval_metric,hyperparameters=hyperparameters,problem_type=problem_type,model_master='hnt',model_branch='fzrt',model_root=model_path) 
hnt_zrt_predictor = train_model(hnt_zrt_data,label=label,eval_metric=eval_metric,hyperparameters=hyperparameters,problem_type=problem_type,model_master='hnt',model_branch='zrt',model_root=model_path) 
# 泥页岩
nyy_sdt_predictor = train_model(nyy_sdt_data,label=label,eval_metric=eval_metric,hyperparameters=hyperparameters,problem_type=problem_type,model_master='nyy',model_branch='sdt',model_root=model_path) 
nyy_fzrt_predictor = train_model(nyy_fzrt_data,label=label,eval_metric=eval_metric,hyperparameters=hyperparameters,problem_type=problem_type,model_master='nyy',model_branch='fzrt',model_root=model_path) 
nyy_zrt_predictor = train_model(nyy_zrt_data,label=label,eval_metric=eval_metric,hyperparameters=hyperparameters,problem_type=problem_type,model_master='nyy',model_branch='zrt',model_root=model_path) 
# 紫红色砂页岩
zhsyy_sdt_predictor = train_model(zhsyy_sdt_data,label=label,eval_metric=eval_metric,hyperparameters=hyperparameters,problem_type=problem_type,model_master='zhsyy',model_branch='sdt',model_root=model_path) 
zhsyy_fzrt_predictor = train_model(zhsyy_fzrt_data,label=label,eval_metric=eval_metric,hyperparameters=hyperparameters,problem_type=problem_type,model_master='zhsyy',model_branch='fzrt',model_root=model_path) 
zhsyy_zrt_predictor = train_model(zhsyy_zrt_data,label=label,eval_metric=eval_metric,hyperparameters=hyperparameters,problem_type=problem_type,model_master='zhsyy',model_branch='zrt',model_root=model_path) 
# 河流冲积物
hlcjw_sdt_predictor = train_model(hlcjw_sdt_data,label=label,eval_metric=eval_metric,hyperparameters=hyperparameters,problem_type=problem_type,model_master='hlcjw',model_branch='sdt',model_root=model_path)
hlcjw_fzrt_predictor = train_model(hlcjw_fzrt_data,label=label,eval_metric=eval_metric,hyperparameters=hyperparameters,problem_type=problem_type,model_master='hlcjw',model_branch='fzrt',model_root=model_path)

# 砂页岩
syy_sdt_predictor = train_model(syy_sdt_data,label=label,eval_metric=eval_metric,hyperparameters=hyperparameters,problem_type=problem_type,model_master='syy',model_branch='sdt',model_root=model_path) 
syy_fzrt_predictor = train_model(syy_fzrt_data,label=label,eval_metric=eval_metric,hyperparameters=hyperparameters,problem_type=problem_type,model_master='syy',model_branch='fzrt',model_root=model_path) 
syy_zrt_predictor = train_model(syy_zrt_data,label=label,eval_metric=eval_metric,hyperparameters=hyperparameters,problem_type=problem_type,model_master='syy',model_branch='zrt',model_root=model_path) 

No presets specified! To achieve strong results with AutoGluon, it is recommended to use the available presets.
	Recommended Presets (For more details refer to https://auto.gluon.ai/stable/tutorials/tabular/tabular-essentials.html#presets):
	presets='best_quality'   : Maximize accuracy. Default time_limit=3600.
	presets='high_quality'   : Strong accuracy with fast inference speed. Default time_limit=3600.
	presets='good_quality'   : Good accuracy with very fast inference speed. Default time_limit=3600.
	presets='medium_quality' : Fast training time, ideal for initial prototyping.
Beginning AutoGluon training ...
AutoGluon will save models to "F:\cache_data\zone_ana\sb\modle\autogluon_type_best\sy\sdt"
=================== System Info ===================
AutoGluon Version:  1.1.0
Python Version:     3.10.11
Operating System:   Windows
Platform Machine:   AMD64
Platform Version:   10.0.22631
CPU Count:          20
Memory Avail:       45.51 GB / 63.81 GB (71.3%)
Disk Space Avail:   374.67 

*** Summary of fit() ***
Estimated performance of each model:
                 model  score_val  eval_metric  pred_time_val  fit_time  pred_time_val_marginal  fit_time_marginal  stack_level  can_infer  fit_order
0  WeightedEnsemble_L2   0.677778  f1_weighted       0.012761  1.290998                0.000000           0.086412            2       True          7
1      NeuralNetFastAI   0.649152  f1_weighted       0.006644  0.560973                0.006644           0.560973            1       True          1
2       NeuralNetTorch   0.556818  f1_weighted       0.006117  0.643612                0.006117           0.643612            1       True          6
3       ExtraTreesEntr   0.520513  f1_weighted       0.030981  0.362455                0.030981           0.362455            1       True          5
4     RandomForestEntr   0.464962  f1_weighted       0.023525  0.360192                0.023525           0.360192            1       True          3
5       ExtraTreesGini   0.464962  f1_

	0.6667	 = Validation score   (f1_weighted)
	0.26s	 = Training   runtime
	0.0s	 = Validation runtime
Fitting model: RandomForestGini ...
	0.4667	 = Validation score   (f1_weighted)
	0.35s	 = Training   runtime
	0.03s	 = Validation runtime
Fitting model: RandomForestEntr ...
	0.3955	 = Validation score   (f1_weighted)
	0.36s	 = Training   runtime
	0.03s	 = Validation runtime
Fitting model: ExtraTreesGini ...
	0.3953	 = Validation score   (f1_weighted)
	0.37s	 = Training   runtime
	0.03s	 = Validation runtime
Fitting model: ExtraTreesEntr ...
	0.3953	 = Validation score   (f1_weighted)
	0.33s	 = Training   runtime
	0.02s	 = Validation runtime
Fitting model: NeuralNetTorch ...
	0.6154	 = Validation score   (f1_weighted)
	0.21s	 = Training   runtime
	0.01s	 = Validation runtime
Fitting model: WeightedEnsemble_L2 ...
	Ensemble Weights: {'NeuralNetFastAI': 1.0}
	0.6667	 = Validation score   (f1_weighted)
	0.09s	 = Training   runtime
	0.0s	 = Validation runtime
AutoGluon training complete, to

*** Summary of fit() ***
Estimated performance of each model:
                 model  score_val  eval_metric  pred_time_val  fit_time  pred_time_val_marginal  fit_time_marginal  stack_level  can_infer  fit_order
0  WeightedEnsemble_L2   0.666667  f1_weighted       0.000000  0.349140                0.000000           0.085215            2       True          7
1      NeuralNetFastAI   0.666667  f1_weighted       0.000000  0.263926                0.000000           0.263926            1       True          1
2       NeuralNetTorch   0.615385  f1_weighted       0.005008  0.206553                0.005008           0.206553            1       True          6
3     RandomForestGini   0.466667  f1_weighted       0.031035  0.353425                0.031035           0.353425            1       True          2
4     RandomForestEntr   0.395455  f1_weighted       0.033589  0.356607                0.033589           0.356607            1       True          3
5       ExtraTreesEntr   0.395294  f1_

	0.6681	 = Validation score   (f1_weighted)
	0.31s	 = Training   runtime
	0.0s	 = Validation runtime
Fitting model: RandomForestGini ...
	0.8181	 = Validation score   (f1_weighted)
	0.41s	 = Training   runtime
	0.04s	 = Validation runtime
Fitting model: RandomForestEntr ...
	0.8268	 = Validation score   (f1_weighted)
	0.42s	 = Training   runtime
	0.03s	 = Validation runtime
Fitting model: ExtraTreesGini ...
	0.8273	 = Validation score   (f1_weighted)
	0.42s	 = Training   runtime
	0.03s	 = Validation runtime
Fitting model: ExtraTreesEntr ...
	0.8273	 = Validation score   (f1_weighted)
	0.46s	 = Training   runtime
	0.04s	 = Validation runtime
Fitting model: NeuralNetTorch ...
	0.7407	 = Validation score   (f1_weighted)
	3.8s	 = Training   runtime
	0.01s	 = Validation runtime
Fitting model: WeightedEnsemble_L2 ...
	Ensemble Weights: {'ExtraTreesGini': 0.5, 'ExtraTreesEntr': 0.5}
	0.8356	 = Validation score   (f1_weighted)
	0.09s	 = Training   runtime
	0.0s	 = Validation runtime
AutoGluon 

...

*** Summary of fit() ***
Estimated performance of each model:
                 model  score_val  eval_metric  pred_time_val  fit_time  pred_time_val_marginal  fit_time_marginal  stack_level  can_infer  fit_order
0  WeightedEnsemble_L2   0.711264  f1_weighted       0.005019  0.799157                0.000000           0.089155            2       True          7
1      NeuralNetFastAI   0.711264  f1_weighted       0.005019  0.710002                0.005019           0.710002            1       True          1
2     RandomForestGini   0.639257  f1_weighted       0.029735  0.368991                0.029735           0.368991            1       True          2
3       NeuralNetTorch   0.632384  f1_weighted       0.006504  0.583560                0.006504           0.583560            1       True          6
4     RandomForestEntr   0.606931  f1_weighted       0.029291  0.355258                0.029291           0.355258            1       True          3
5       ExtraTreesEntr   0.603656  f1_

No improvement since epoch 0: early stopping
	0.7511	 = Validation score   (f1_weighted)
	0.21s	 = Training   runtime
	0.0s	 = Validation runtime
Fitting model: RandomForestGini ...
	0.7511	 = Validation score   (f1_weighted)
	0.36s	 = Training   runtime
	0.03s	 = Validation runtime
Fitting model: RandomForestEntr ...
	0.7511	 = Validation score   (f1_weighted)
	0.34s	 = Training   runtime
	0.03s	 = Validation runtime
Fitting model: ExtraTreesGini ...
	0.6193	 = Validation score   (f1_weighted)
	0.35s	 = Training   runtime
	0.04s	 = Validation runtime
Fitting model: ExtraTreesEntr ...
	0.6193	 = Validation score   (f1_weighted)
	0.34s	 = Training   runtime
	0.03s	 = Validation runtime
Fitting model: NeuralNetTorch ...
	0.6193	 = Validation score   (f1_weighted)
	0.38s	 = Training   runtime
	0.01s	 = Validation runtime
Fitting model: WeightedEnsemble_L2 ...
	Ensemble Weights: {'NeuralNetFastAI': 1.0}
	0.7511	 = Validation score   (f1_weighted)
	0.09s	 = Training   runtime
	0.0s	 = Valid

*** Summary of fit() ***
Estimated performance of each model:
                 model  score_val  eval_metric  pred_time_val  fit_time  pred_time_val_marginal  fit_time_marginal  stack_level  can_infer  fit_order
0  WeightedEnsemble_L2   0.751131  f1_weighted       0.001917  0.294357                0.000000           0.085222            2       True          7
1      NeuralNetFastAI   0.751131  f1_weighted       0.001917  0.209134                0.001917           0.209134            1       True          1
2     RandomForestEntr   0.751131  f1_weighted       0.033022  0.340857                0.033022           0.340857            1       True          3
3     RandomForestGini   0.751131  f1_weighted       0.033050  0.355690                0.033050           0.355690            1       True          2
4       NeuralNetTorch   0.619263  f1_weighted       0.014844  0.378475                0.014844           0.378475            1       True          6
5       ExtraTreesEntr   0.619263  f1_

	0.6827	 = Validation score   (f1_weighted)
	0.31s	 = Training   runtime
	0.0s	 = Validation runtime
Fitting model: RandomForestGini ...
	0.7496	 = Validation score   (f1_weighted)
	0.36s	 = Training   runtime
	0.03s	 = Validation runtime
Fitting model: RandomForestEntr ...
	0.7664	 = Validation score   (f1_weighted)
	0.35s	 = Training   runtime
	0.03s	 = Validation runtime
Fitting model: ExtraTreesGini ...
	0.7558	 = Validation score   (f1_weighted)
	0.36s	 = Training   runtime
	0.03s	 = Validation runtime
Fitting model: ExtraTreesEntr ...
	0.7664	 = Validation score   (f1_weighted)
	0.36s	 = Training   runtime
	0.03s	 = Validation runtime
Fitting model: NeuralNetTorch ...
	0.6763	 = Validation score   (f1_weighted)
	1.49s	 = Training   runtime
	0.01s	 = Validation runtime
Fitting model: WeightedEnsemble_L2 ...
	Ensemble Weights: {'RandomForestEntr': 0.5, 'ExtraTreesGini': 0.5}
	0.7814	 = Validation score   (f1_weighted)
	0.09s	 = Training   runtime
	0.0s	 = Validation runtime
AutoGlu

*** Summary of fit() ***
Estimated performance of each model:
                 model  score_val  eval_metric  pred_time_val  fit_time  pred_time_val_marginal  fit_time_marginal  stack_level  can_infer  fit_order
0  WeightedEnsemble_L2   0.781400  f1_weighted       0.056231  0.798092                0.000000           0.086489            2       True          7
1     RandomForestEntr   0.766414  f1_weighted       0.026980  0.353143                0.026980           0.353143            1       True          3
2       ExtraTreesEntr   0.766414  f1_weighted       0.032291  0.361037                0.032291           0.361037            1       True          5
3       ExtraTreesGini   0.755799  f1_weighted       0.029252  0.358459                0.029252           0.358459            1       True          4
4     RandomForestGini   0.749648  f1_weighted       0.033442  0.358901                0.033442           0.358901            1       True          2
5      NeuralNetFastAI   0.682723  f1_